In [1]:
# NLP Lecture @ Strive School - 21st July 2021
# CHATBOT with Pytorch

'''
Chatbots are essential for speeding up user assistance and reducing waiting times. Chatbots can quickly extract important information such as demographics, symptoms, health insurance information and assist any patients by making appointments with specialists.

Imagine having to design a tool that allows preliminary assistance for those who must access a treatment path or must make a reservation for a specialist visit.

Create a dataset using the template provided as a base and prepare at least 5 different intents with 4/5 responses each.

The final result must ensure that users can have a dialogue of at least 3 questions and 3 answers consistent with the context.

Example
A: Hello MedAssistant.
B: Hello. How can I help you?
A: I don't feel well.
B: Do you have any symptoms?
A: I have cough and nausea.
B: Do you want to book an appointment?
A: Yes, please, for tomorrow.


Info:
- Feel free to change or arrange a new dataset of intents
- Try experimenting and tuning with the hyperparameters
- Feel free to use or change the code you've seen during the morning session
- TBD = To be done (from you!) :)
'''

"\nChatbots are essential for speeding up user assistance and reducing waiting times. Chatbots can quickly extract important information such as demographics, symptoms, health insurance information and assist any patients by making appointments with specialists.\n\nImagine having to design a tool that allows preliminary assistance for those who must access a treatment path or must make a reservation for a specialist visit.\n\nCreate a dataset using the template provided as a base and prepare at least 5 different intents with 4/5 responses each.\n\nThe final result must ensure that users can have a dialogue of at least 3 questions and 3 answers consistent with the context.\n\nExample\nA: Hello MedAssistant.\nB: Hello. How can I help you?\nA: I don't feel well.\nB: Do you have any symptoms?\nA: I have cough and nausea.\nB: Do you want to book an appointment?\nA: Yes, please, for tomorrow.\n\n\nInfo:\n- Feel free to change or arrange a new dataset of intents\n- Try experimenting and tunin

In [2]:
import numpy as np
import json

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import random

from nltk_utils import bag_of_words, tokenize, stem
from model import NeuralNet

# STEP 0: find intents patterns

with open('medical_intents.json', 'r') as f:
    intents = json.load(f)

In [3]:
# STEP 1: Pre-process of the input

# lower case? stemming? stopwords?
# TBD
import spacy

nlp = spacy.load("en_core_web_sm")

def preprocess(text):
    #remove punctuation from text and remove stop words from text
    text = ' '.join([token.lemma_ for token in nlp(text) if not token.is_punct])
    return text

all_words = []
tags = []
patterns = []

for intent in intents['intents']:
# TBD: loop through each sentence in our intents patterns, create a list of tags and define the patterns
    tags.append(intent['tag'])
    for pattern in intent['patterns']:
        pattern = preprocess(pattern)
        words = nlp(pattern)
        patterns.append(([token.text for token in words],intent['tag']))
        all_words.extend(words)
all_words = sorted(set([word.text for word in all_words]))

In [4]:
# STEP 2: Define training data through a bag of words

X_train = []
y_train = []
for (pattern_sentence, tag) in patterns:
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
    label = tags.index(tag)
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)

In [5]:
arg1 = patterns[0][0]
print(arg1)
dic = {}
for i,word in enumerate(all_words):
    dic[word] = bag_of_words(arg1, all_words)[i]
dic

['hi']


{'  ': 0.0,
 '24': 0.0,
 '3': 0.0,
 '37.8c': 0.0,
 'I': 0.0,
 'a': 0.0,
 'abdomen': 0.0,
 'abdominal': 0.0,
 'activity': 0.0,
 'alcohol': 0.0,
 'an': 0.0,
 'and': 0.0,
 'ankle': 0.0,
 'anxiety': 0.0,
 'anyone': 0.0,
 'appetite': 0.0,
 'as': 0.0,
 'ascite': 0.0,
 'attack': 0.0,
 'bad': 0.0,
 'be': 0.0,
 'black': 0.0,
 'bleed': 0.0,
 'bleeding': 0.0,
 'block': 0.0,
 'blocked': 0.0,
 'blood': 0.0,
 'blotchy': 0.0,
 'brain': 0.0,
 'breath': 0.0,
 'bruise': 0.0,
 'build': 0.0,
 'call': 0.0,
 'can': 0.0,
 'change': 0.0,
 'chest': 0.0,
 'club': 0.0,
 'confusion': 0.0,
 'continuous': 0.0,
 'cough': 0.0,
 'crack': 0.0,
 'curve': 0.0,
 'day': 0.0,
 'delusion': 0.0,
 'depression': 0.0,
 'diarrhoea': 0.0,
 'disinter': 0.0,
 'dizziness': 0.0,
 'drug': 0.0,
 'dry': 0.0,
 'due': 0.0,
 'easily': 0.0,
 'episode': 0.0,
 'eye': 0.0,
 'face': 0.0,
 'fatigue': 0.0,
 'feel': 0.0,
 'fever': 0.0,
 'finger': 0.0,
 'fingertip': 0.0,
 'flashback': 0.0,
 'fluid': 0.0,
 'foot': 0.0,
 'for': 0.0,
 'frequent': 0.0,


In [6]:
from collections import OrderedDict
import torch.nn.functional as F
hidden_sizes = [88,36,len(tags)*2]
class Network(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_sizes[0])
        self.fc2 = nn.Linear(hidden_sizes[0], hidden_sizes[1])
        self.fc3 = nn.Linear(hidden_sizes[1], hidden_sizes[2])
        self.fc4 = nn.Linear(hidden_sizes[2], output_size)
        self.dropout = nn.Dropout(p=0.15)
        
    # Forward pass through the network, returns the output logits
    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.fc3(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.fc4(x)
        x = F.relu(x)
        x = self.dropout(x)
        return x

    

In [7]:
# STEP 3: Configure the neural network

# define each parameter that is equal to 0 using an empirical value or a value based on your experience
# TBD


batch_size = 8
learning_rate = 0.001
input_size = len(X_train[0])
hidden_size = 12
output_size = len(tags)

# STEP 4: Train the model

class ChatDataset(Dataset):

    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples

dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=0)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = NeuralNet(input_size, hidden_size, output_size).to(device)
#model = Network().to(device)

In [8]:
words, labels = iter(train_loader).next()

In [9]:
# Define loss and optimizer: which one is the best one?
# TBD

num_epochs = 2000
learning_rate = 0.001

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)



# Train the model
for epoch in range(num_epochs):
    loss_e = 0
    for i,(words, labels) in enumerate(train_loader):
        words = words.to(device)
        labels = labels.to(dtype=torch.long).to(device)

        # Forward pass
        outputs = model(words)
        # if y would be one-hot, we must apply
        # labels = torch.max(labels, 1)[1]
        loss = criterion(outputs, labels)
        loss_e += loss.item()

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if i+1 % 5 == 0:
            print(f'Epoch {epoch+1}/{num_epochs}: loss {loss_e/(i+1):.3f}')


data = {
    "model_state": model.state_dict(),
    "input_size": input_size,
    "hidden_size": hidden_size,
    "output_size": output_size,
    "all_words": all_words,
    "tags": tags
}

In [10]:
# STEP 5: Save the model
# TBD: name and save the model
torch.save(data ,'chat_data.pth')

In [16]:
# STEP 6: Test the model

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# TBD: Load the intents file
with open('medical_intents.json', 'r') as f:
    intents = json.load(f)

# TBD: Retrieve the model and all the sizings
data = torch.load('chat_data.pth')
model_state_dict = data["model_state"]
input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data["all_words"]
tags = data["tags"]


# TBD: build the NN
model = NeuralNet(input_size, hidden_size, output_size).to(device)
#model = Network().to(device) 
model.load_state_dict(model_state_dict)
model.eval()

# TBD: prepare a command-line conversation (don't forget something to make the user exit the script!)
bot_name = "MedAssistant"
print('Welcome to Medservice')
while True:
    sentence = input("You: ")
    if sentence.lower() == "exit":
        print('Hope it was usefull, I\'ll be here!')
        break
    else:
        sentence_processed = preprocess(sentence)
        sentence_processed = [token.text for token in nlp(sentence_processed)]
        X = bag_of_words(sentence_processed, all_words)
        X = torch.from_numpy(X).to(device)
        
        output = model(X)
        _, predicted = torch.max(output, dim=0)

        tag = tags[predicted.item()]
        probs = torch.softmax(output, dim=0)
        prob = probs[predicted.item()]
        
        if prob.item()>0.75:
            for intent in intents['intents']:
                if intent['tag'] == tag:
                    print(f'{bot_name}: {random.choice(intent["responses"])}')
                    
        elif sum(torch.softmax(output, dim=0)>0.4).item() > 1:
            possible = np.array(tags)[(torch.softmax(output, dim=0)>0.4).cpu().detach().numpy()]
            print(f'{bot_name}: I\'m not sure if you have {possible[0]} or {possible[1]}. Try to say me some extra symptoms please!')
        
        else:
            print(f'{bot_name}: Try to be more specific please!')


Welcome to Medservice
You: hi bot
MedAssistant: Hi there, how can I help?
You: whats your name
MedAssistant: I'm bot!
You: im feeling bad
MedAssistant: which are your principal symptons?
You: a cough
MedAssistant: I'm not sure if you have COVID-19 or common cold. Try to say me some extra symptoms please!
You: im coughing a lot
MedAssistant: I'm not sure if you have COVID-19 or common cold. Try to say me some extra symptoms please!
You: i think im coughing more than usual
MedAssistant: you might be suffering from COVID-19
You: exit
Hope it was usefull, I'll be here!
